In [1]:
import os

import boto3
import pandas as pd
import pyspark

In [2]:
session = boto3.Session(
    aws_access_key_id = os.environ.get("MINIO_ROOT_USER"),
    aws_secret_access_key = os.environ.get("MINIO_ROOT_PASSWORD"),
)
s3 = session.resource('s3', 
  endpoint_url="http://minio:9000", 
  config=boto3.session.Config(signature_version='s3v4')
)
s3.create_bucket(Bucket='spark-data')

BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [37]:
sc.stop()

In [38]:
conf = pyspark.SparkConf().setMaster("spark://spark-master:7077")
conf.set('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000')
conf.set('spark.hadoop.fs.s3a.access.key', os.environ.get("MINIO_ROOT_USER"))
conf.set('spark.hadoop.fs.s3a.secret.key', os.environ.get("MINIO_ROOT_PASSWORD"))
conf.set('spark.hadoop.fs.s3a.path.style.access', "true")
conf.set("spark.jars.packages", "io.delta:delta-core_2.12-3.2.1")
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
sc = pyspark.SparkContext(conf=conf)

In [7]:
spark.stop()

24/11/27 16:45:19 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/11/27 16:45:19 INFO SparkUI: Stopped Spark web UI at http://77ad02ff99d3:4040
24/11/27 16:45:19 INFO StandaloneSchedulerBackend: Shutting down all executors
24/11/27 16:45:19 INFO StandaloneSchedulerBackend$StandaloneDriverEndpoint: Asking each executor to shut down
24/11/27 16:45:19 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/11/27 16:45:19 INFO MemoryStore: MemoryStore cleared
24/11/27 16:45:19 INFO BlockManager: BlockManager stopped
24/11/27 16:45:19 INFO BlockManagerMaster: BlockManagerMaster stopped
24/11/27 16:45:19 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/11/27 16:45:19 INFO SparkContext: Successfully stopped SparkContext


In [8]:
spark = pyspark.sql.SparkSession.builder. \
  config('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000'). \
  config('spark.hadoop.fs.s3a.access.key', os.environ.get("MINIO_ROOT_USER")). \
  config('spark.hadoop.fs.s3a.secret.key', os.environ.get("MINIO_ROOT_PASSWORD")). \
  config('spark.hadoop.fs.s3a.path.style.access', "true"). \
  config('spark.jars', "/opt/spark_jars/delta-spark_2.13-3.2.0.jar"). \
  config("spark.jars.packages", "io.delta:delta-spark_2.13:3.2.0"). \
  config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension"). \
  master("spark://spark-master:7077").getOrCreate()

24/11/27 16:45:21 INFO SparkContext: Running Spark version 3.5.3
24/11/27 16:45:21 INFO SparkContext: OS info Linux, 6.8.0-49-generic, amd64
24/11/27 16:45:21 INFO SparkContext: Java version 17.0.13
24/11/27 16:45:21 INFO ResourceUtils: ==============================================================
24/11/27 16:45:21 INFO ResourceUtils: No custom resources configured for spark.driver.
24/11/27 16:45:21 INFO ResourceUtils: ==============================================================
24/11/27 16:45:21 INFO SparkContext: Submitted application: pyspark-shell
24/11/27 16:45:21 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
24/11/27 16:45:21 INFO ResourceProfile: Limiting resource is cpu
24/11/27 16:45:21 INFO ResourceProfileManager: Added ResourceProfile id: 0
24/11/27 16:45:21 INFO SecurityM

In [9]:
transactions = pd.DataFrame({
    "id": [i for i in range(10)],
    "date": ["2024-01-01"] * 4 + ["2024-02-01"] * 6,
    "cust_id": [0, 0, 0, 1, 1, 2, 2, 2, 3, 3],
    "prod_id": [0, 1, 2, 0, 1, 2, 2, 3, 0, 3],
    "volume": [1, 1, 1, 1, 2, 2, 3, 3, 4, 4],
    "revenue": [10, 15, 40, 10, 30, 80, 120, 150, 40, 160]
}).set_index("id")

In [10]:
transactions_spark = spark.createDataFrame(transactions)

24/11/27 16:45:44 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/11/27 16:45:44 INFO SharedState: Warehouse path is 'file:/home/jptr/app/notebooks/spark-warehouse'.


In [11]:
transactions_spark.show()

24/11/27 16:45:54 INFO CodeGenerator: Code generated in 81.825057 ms
24/11/27 16:45:54 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/11/27 16:45:54 INFO DAGScheduler: Got job 0 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/11/27 16:45:54 INFO DAGScheduler: Final stage: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0)
24/11/27 16:45:54 INFO DAGScheduler: Parents of final stage: List()
24/11/27 16:45:54 INFO DAGScheduler: Missing parents: List()
24/11/27 16:45:54 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[6] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/11/27 16:45:54 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 13.9 KiB, free 434.4 MiB)
24/11/27 16:45:55 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 7.0 KiB, free 434.4 MiB)
24/11/27 16:45:55 INFO BlockManagerInfo: Added broadcast_0_

Py4JJavaError: An error occurred while calling o146.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.18.0.5 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 8) than that in driver 3.12, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner

24/11/27 16:52:46 INFO BlockManagerInfo: Removed broadcast_0_piece0 on 77ad02ff99d3:33423 in memory (size: 7.0 KiB, free: 434.4 MiB)
24/11/27 16:52:46 INFO BlockManagerInfo: Removed broadcast_0_piece0 on 172.18.0.5:41527 in memory (size: 7.0 KiB, free: 434.4 MiB)
24/11/27 16:52:46 INFO BlockManagerInfo: Removed broadcast_0_piece0 on 172.18.0.6:43385 in memory (size: 7.0 KiB, free: 434.4 MiB)


In [ ]:
db_url = "jdbc:postgresql://demo_db:5432/demo_db"
properties = {
    "user": "user",
    "password": "shhh_its_a_secret",
    "driver": "org.postgresql.Driver"
}